# Testabfragen

Dies sind Testabfragen für die DNB-API. Es wird eine Abfrage für MARC21-XML und eine für RDF-XML durchgeführt. Die Ergebnisse werden in Tabellenform dargestellt und als CSV-Datei gespeichert.

Die Abfragen basieren auf dem Tutorial DNB_Testabfrage_SRU.ipynb.

## MARC21-XML Abfrage

In [1]:
import requests
from IPython.display import display, clear_output
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

import pprint

In [2]:
def dnb_sru(query):

    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
              'operation': 'searchRetrieve',
              'version': '1.1',
              'maximumRecords': '100',
              'query': query
              }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})

    if len(records) < 100:

        return records

    else:

        num_results = 100
        i = 101
        while num_results == 100:

            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)

        return records

In [3]:
def parse_record(record):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'

    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)

    try:
        titel = titel[0].text
        #umfang = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"

        # Archivlink
    urn = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)

    try:
        urn = urn[0].text
        #link = unicodedata.normalize("NFC", urn)
    except:
        urn = 'fail'

        # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = 'fail'

    meta_dict = {"idn":idn,
                 "titel":titel,
                 }

    return meta_dict

In [4]:
records = dnb_sru('tit="Jeshurun"')
print(len(records), 'Ergebnisse')

0 Ergebnisse


/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [9]:
print(records)

[<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
<leader>00000naa a2200000uc 4500</leader>
<controlfield tag="001">1039499449</controlfield>
<controlfield tag="003">DE-101</controlfield>
<controlfield tag="005">20171203124355.0</controlfield>
<controlfield tag="007">cr||||||||||||</controlfield>
<controlfield tag="008">130830s1941    gw |||||o|||| 00|||||||  </controlfield>
<datafield ind1="7" ind2=" " tag="016">
<subfield code="2">DE-101</subfield>
<subfield code="a">1039499449</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="035">
<subfield code="a">(DE-599)DNB1039499449</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="035">
<subfield code="a">(OCoLC)1185617069</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="040">
<subfield code="a">9999</subfield>
<subfield code="b">ger</subfield>
<subfield code="c">DE-101</subfield>
<subfield code="d">9999</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="044">
<subfield code="c">XA-DE</subfi

In [10]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df


,idn,titel
0,1039499449,Adath Jeshurun of West Bronx
1,1039336116,Aus den Gemeinden
2,1039504418,Aus den Gemeinden
3,103908205X,Aus den Gemeinden
4,1039273734,Aus den Gemeinden
5,1039438938,Aus den Gemeinden
6,1039320953,Aus den Gemeinden
7,1039385184,B'nai Jeshurun beschliesst ihr Jubiläum
8,1039376231,Congregation B'nai Jeshurun - 125 Jahre
9,103949823X,K'Hal Adath Jeshurun


## RDF-XML Abfrage

In [42]:
def dnb_sru_rdf(query):

    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'RDFxml',
              'operation': 'searchRetrieve',
              'version': '1.1',
              'maximumRecords': '100',
              'query': query
              }
    r = requests.get(base_url, params=params)
    records = soup(r.content)

    return r.content

In [12]:
def parse_record_rdf(record):

    ns = {"xlmns":"http://www.loc.gov/zing/srw/",
          "agrelon":"https://d-nb.info/standards/elementset/agrelon#",
          "bflc":"http://id.loc.gov/ontologies/bflc/",
          "rdau":"http://rdaregistry.info/Elements/u/",
          "dc":"http://purl.org/dc/elements/1.1/",
          "rdau":"http://rdaregistry.info/Elements/u",
          "bibo":"http://purl.org/ontology/bibo/",
          "dbp":"http://dbpedia.org/property/",
          "dcmitype":"http://purl.org/dc/dcmitype/",
          "dcterms":"http://purl.org/dc/terms/",
          "dnb_intern":"http://dnb.de/",
          "dnbt":"https://d-nb.info/standards/elementset/dnb#",
          "ebu":"http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#",
          "editeur":"https://ns.editeur.org/thema/",
          "foaf":"http://xmlns.com/foaf/0.1/",
          "gbv":"http://purl.org/ontology/gbv/",
          "geo":"http://www.opengis.net/ont/geosparql#",
          "gndo":"https://d-nb.info/standards/elementset/gnd#",
          "isbd":"http://iflastandards.info/ns/isbd/elements/",
          "lib":"http://purl.org/library/",
          "madsrdf":"http://www.loc.gov/mads/rdf/v1#",
          "marcrole":"http://id.loc.gov/vocabulary/relators/",
          "mo":"http://purl.org/ontology/mo/",
          "owl":"http://www.w3.org/2002/07/owl#",
          "rdf":"http://www.w3.org/1999/02/22-rdf-syntax-ns#",
          "rdfs":"http://www.w3.org/2000/01/rdf-schema#",
          "schema":"http://schema.org/",
          "sf":"http://www.opengis.net/ont/sf#",
          "skos":"http://www.w3.org/2004/02/skos/core#",
          "umbel":"http://umbel.org/umbel#",
          "v":"http://www.w3.org/2006/vcard/ns#",
          "vivo":"http://vivoweb.org/ontology/core#",
          "wdrs":"http://www.w3.org/2007/05/powder-s#",
          "xsd":"http://www.w3.org/2001/XMLSchema#"}

    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    #idn
    idn = xml.findall(".//dc:identifier", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A'


        #creator
    creator = record.find_all('rdau:p60327')

    try:
        creator = creator[0].text
    except:
        creator = "N/A"


    #title
    test = record.find_all('dc:title')

    try:
        test = test[0].text
    except:
        test = "N/A"


    #date
    date = record.find_all('dcterms:issued')

    try:
        date = date[0].text
    except:
        date = "N/A"


        #publisher
    publ = record.find_all('dc:publisher')

    try:
        publ = publ[0].text
    except:
        publ = "N/A"

        #urn
    urn = record.find_all('umbel:islike')

    try:
        urn = urn[0]
        urn = urn.get('rdf:resource')
    except:
        urn = "N/A"

    #isbn
    isbn = xml.findall(".//bibo:isbn13", namespaces=ns)
    isbn10 = xml.findall(".//bibo:isbn10", namespaces=ns)

    if isbn:
        isbn = isbn[0].text
    elif isbn10:
        isbn = isbn10[0].text
    else:
        isbn = "N/A"




    meta_dict = {"IDN":idn, "CREATOR":creator, "TITLE":test, "DATE":date, "PUBLISHER":publ, "URN":urn, "ISBN":isbn}

    return meta_dict

In [14]:
#GND in RDF

def parse_record_gndrdf(record):

    ns = {"xlmns":"http://www.loc.gov/zing/srw/",
          "agrelon":"https://d-nb.info/standards/elementset/agrelon#",
          "bflc":"http://id.loc.gov/ontologies/bflc/",
          "rdau":"http://rdaregistry.info/Elements/u/",
          "dc":"http://purl.org/dc/elements/1.1/",
          "rdau":"http://rdaregistry.info/Elements/u",
          "bibo":"http://purl.org/ontology/bibo/",
          "dbp":"http://dbpedia.org/property/",
          "dcmitype":"http://purl.org/dc/dcmitype/",
          "dcterms":"http://purl.org/dc/terms/",
          "dnb_intern":"http://dnb.de/",
          "dnbt":"https://d-nb.info/standards/elementset/dnb#",
          "ebu":"http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#",
          "editeur":"https://ns.editeur.org/thema/",
          "foaf":"http://xmlns.com/foaf/0.1/",
          "gbv":"http://purl.org/ontology/gbv/",
          "geo":"http://www.opengis.net/ont/geosparql#",
          "gndo":"https://d-nb.info/standards/elementset/gnd#",
          "isbd":"http://iflastandards.info/ns/isbd/elements/",
          "lib":"http://purl.org/library/",
          "madsrdf":"http://www.loc.gov/mads/rdf/v1#",
          "marcrole":"http://id.loc.gov/vocabulary/relators/",
          "mo":"http://purl.org/ontology/mo/",
          "owl":"http://www.w3.org/2002/07/owl#",
          "rdf":"http://www.w3.org/1999/02/22-rdf-syntax-ns#",
          "rdfs":"http://www.w3.org/2000/01/rdf-schema#",
          "schema":"http://schema.org/",
          "sf":"http://www.opengis.net/ont/sf#",
          "skos":"http://www.w3.org/2004/02/skos/core#",
          "umbel":"http://umbel.org/umbel#",
          "v":"http://www.w3.org/2006/vcard/ns#",
          "vivo":"http://vivoweb.org/ontology/core#",
          "wdrs":"http://www.w3.org/2007/05/powder-s#",
          "xsd":"http://www.w3.org/2001/XMLSchema#"}

    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    #idn
    idn = xml.findall(".//gndo:gndidentifier", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A'


        #link
    link = record.find_all('rdf:description')

    try:
        link = link[0]
        link = link.get('rdf:about')
    except:
        link = 'N/A'


        #name
    name = record.find_all('gndo:preferrednamefortheperson')
    name2 = record.find_all('gndo:preferrednameforthecorporatebody')
    name3 = record.find_all('gndo:preferrednameforthework')
    name4 = record.find_all('gndo:preferrednamefortheconferenceorevent')
    name5 = record.find_all('gndo:preferrednameforthesubjectheading')

    if name:
        name = name[0].text
    elif name2:
        name = name2[0].text
    elif name3:
        name = name3[0].text
    elif name4:
        name = name4[0].text
    elif name5:
        name = name5[0].text
    else:
        name = "N/A"


    #time
    time = record.find_all('gndo:periodofactivity')
    time2 = record.find_all('gndo:dateofpublication')
    time3 = record.find_all('gndo:dateofconferenceorevent')
    time4 = record.find_all('gndo:dateofbirth')

    if time:
        time = time[0].text
    elif time2:
        time = time2[0].text
    elif time3:
        time = time3[0].text
    elif time4:
        time = time4[0].text + "-"
    else:
        time = "N/A"



    #type
    gndtype = record.find_all('rdf:type')

    try:
        gndtype = gndtype[0]
        gndtype = gndtype.get('rdf:resource')
    except:
        link = 'N/A'



    meta_dict = {"IDN":idn, "NAME":name, "TIME":time, "LINK":link}

    return meta_dict




In [ ]:
#DMA in RDF: 

def parse_record_dmardf(record):

    ns = {"xlmns":"http://www.loc.gov/zing/srw/",
          "agrelon":"https://d-nb.info/standards/elementset/agrelon#",
          "bflc":"http://id.loc.gov/ontologies/bflc/",
          "rdau":"http://rdaregistry.info/Elements/u/",
          "dc":"http://purl.org/dc/elements/1.1/",
          "rdau":"http://rdaregistry.info/Elements/u",
          "bibo":"http://purl.org/ontology/bibo/",
          "dbp":"http://dbpedia.org/property/",
          "dcmitype":"http://purl.org/dc/dcmitype/",
          "dcterms":"http://purl.org/dc/terms/",
          "dnb_intern":"http://dnb.de/",
          "dnbt":"https://d-nb.info/standards/elementset/dnb#",
          "ebu":"http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#",
          "editeur":"https://ns.editeur.org/thema/",
          "foaf":"http://xmlns.com/foaf/0.1/",
          "gbv":"http://purl.org/ontology/gbv/",
          "geo":"http://www.opengis.net/ont/geosparql#",
          "gndo":"https://d-nb.info/standards/elementset/gnd#",
          "isbd":"http://iflastandards.info/ns/isbd/elements/",
          "lib":"http://purl.org/library/",
          "madsrdf":"http://www.loc.gov/mads/rdf/v1#",
          "marcrole":"http://id.loc.gov/vocabulary/relators/",
          "mo":"http://purl.org/ontology/mo/",
          "owl":"http://www.w3.org/2002/07/owl#",
          "rdf":"http://www.w3.org/1999/02/22-rdf-syntax-ns#",
          "rdfs":"http://www.w3.org/2000/01/rdf-schema#",
          "schema":"http://schema.org/",
          "sf":"http://www.opengis.net/ont/sf#",
          "skos":"http://www.w3.org/2004/02/skos/core#",
          "umbel":"http://umbel.org/umbel#",
          "v":"http://www.w3.org/2006/vcard/ns#",
          "vivo":"http://vivoweb.org/ontology/core#",
          "wdrs":"http://www.w3.org/2007/05/powder-s#",
          "xsd":"http://www.w3.org/2001/XMLSchema#"}

    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    #idn
    idn = xml.findall(".//dc:identifier", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A'


        #creator
    name = record.find_all('rdau:p60327')

    if name:
        name = name[0].text
    else:
        name = "N/A"


    #title:
    title = record.find_all('dc:title')

    if title:
        title = title[0].text
    else:
        title = "N/A"


    #publisher
    publ = record.find_all('dc:publisher')

    if publ:
        publ = publ[0].text
    else:
        publ = "N/A"


    #date
    time = record.find_all('dcterms:issued')

    if time:
        time = time[0].text
    else:
        time = "N/A"




    meta_dict = {"IDN":idn, "NAME":name, "TITLE":title, "PUBLISHER":publ, "DATE":time}

    return meta_dict

In [43]:
query = "tit=Jeshurun"
base_url = "https://services.dnb.de/sru/dnb"
params = {'recordSchema' : 'RDFxml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
          }

records = requests.get(base_url, params=params)

print(records.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/"><version>1.1</version><numberOfRecords>16</numberOfRecords><records><record><recordSchema>RDFxml</recordSchema><recordPacking>xml</recordPacking><recordData><rdf:RDF xmlns:schema="http://schema.org/" xmlns:gndo="https://d-nb.info/standards/elementset/gnd#" xmlns:lib="http://purl.org/library/" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:editeur="https://ns.editeur.org/thema/" xmlns:geo="http://www.opengis.net/ont/geosparql#" xmlns:umbel="http://umbel.org/umbel#" xmlns:naf="https://id.loc.gov/authorities/names/" xmlns:rdau="http://rdaregistry.info/Elements/u/" xmlns:sf="http://www.opengis.net/ont/sf#" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:thesoz="http://lod.gesis.org/thesoz/" xmlns:dcterms="http://purl.org/dc/terms/" xmln

In [44]:
records.text

'<?xml version="1.0" encoding="UTF-8"?>\n<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/"><version>1.1</version><numberOfRecords>16</numberOfRecords><records><record><recordSchema>RDFxml</recordSchema><recordPacking>xml</recordPacking><recordData><rdf:RDF xmlns:schema="http://schema.org/" xmlns:gndo="https://d-nb.info/standards/elementset/gnd#" xmlns:lib="http://purl.org/library/" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:editeur="https://ns.editeur.org/thema/" xmlns:geo="http://www.opengis.net/ont/geosparql#" xmlns:umbel="http://umbel.org/umbel#" xmlns:naf="https://id.loc.gov/authorities/names/" xmlns:rdau="http://rdaregistry.info/Elements/u/" xmlns:sf="http://www.opengis.net/ont/sf#" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:thesoz="http://lod.gesis.org/thesoz/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns

## Test der RDF-Abfrage

In [45]:
records = dnb_sru_rdf('tit=Jeshurun')
print(len(records), 'Ergebnisse')
print(records)

63188 Ergebnisse
b'<?xml version="1.0" encoding="UTF-8"?>\n<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/"><version>1.1</version><numberOfRecords>16</numberOfRecords><records><record><recordSchema>RDFxml</recordSchema><recordPacking>xml</recordPacking><recordData><rdf:RDF xmlns:schema="http://schema.org/" xmlns:gndo="https://d-nb.info/standards/elementset/gnd#" xmlns:lib="http://purl.org/library/" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:editeur="https://ns.editeur.org/thema/" xmlns:geo="http://www.opengis.net/ont/geosparql#" xmlns:umbel="http://umbel.org/umbel#" xmlns:naf="https://id.loc.gov/authorities/names/" xmlns:rdau="http://rdaregistry.info/Elements/u/" xmlns:sf="http://www.opengis.net/ont/sf#" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:thesoz="http://lod.gesis.org/thesoz/" xmlns:dcterms="http://purl.or

/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [37]:
# save the result als xml-file
with open('data.xml', 'w', encoding='utf-8') as f:
    print(records, file=f)

# XXX Create a graph from the RDF-XML results

In [22]:
import re

# Read the RDF/XML file
with open("data.xml", "r") as f:
    rdf_xml = f.read()

# Replace invalid property attribute URI with rdf:about
rdf_xml_fixed = re.sub(r'(?<!rdf:)(about)', 'rdf:about', rdf_xml)

# Write the fixed RDF/XML to a new file
with open("fixed_rdf.xml", "w") as f:
    f.write(rdf_xml_fixed)


In [46]:
from rdflib import Graph

# Parse RDF/XML file
g = Graph()
g.parse("fixed_rdf.xml", format="xml")

# Serialize to Turtle format
turtle_data = g.serialize(format="turtle")

# Print or write to a file
print(turtle_data.decode())  # Decode bytes to string if needed

# Write the Turtle data to a .ttl file
with open("data.ttl", "wb") as f:
    f.write(turtle_data)


ParserError: file:///Users/benni/research/DNB/fixed_rdf.xml:2:0: Invalid property attribute URI: http://www.w3.org/1999/02/22-rdf-syntax-ns#about

# Darstellung in Tabellarischer Form

In [38]:
#Funktion für Links: 
def make_clickable(val):
    if val == "N/A":
        link = "N/A"
    else:
        link = '<a target="_blank" href="{}">{}</a>'.format(val,val)

    return link

## DNB RDF-XML

In [48]:
print(records.text)

AttributeError: 'bytes' object has no attribute 'text'

In [55]:
query = "tit=Jeshurun"
params = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query, 'recordSchema' : "RDFxml", 'maximumRecords': '100'} 

request = requests.get("https://services.dnb.de/sru/dnb", params = params)
response = soup(request.content)
records = response.find_all('record')
print(records)

[<record><recordschema>RDFxml</recordschema><recordpacking>xml</recordpacking><recorddata><rdf:rdf xmlns:agrelon="https://d-nb.info/standards/elementset/agrelon#" xmlns:agrovoc="https://aims.fao.org/aos/agrovoc/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:bibo="http://purl.org/ontology/bibo/" xmlns:cidoc="http://www.cidoc-crm.org/cidoc-crm/" xmlns:dbp="http://dbpedia.org/property/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcmitype="http://purl.org/dc/dcmitype/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:dnb_intern="http://dnb.de/" xmlns:dnbt="https://d-nb.info/standards/elementset/dnb#" xmlns:ebu="http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#" xmlns:editeur="https://ns.editeur.org/thema/" xmlns:embne="https://datos.bne.es/resource/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:gbv="http://purl.org/ontology/gbv/" xmlns:geo="http://www.opengis.net/ont/geosparql#" xmlns:gndo="https://d-nb.info/standards/elementset/gnd#" xmlns:isbd="http://iflastandards.i

/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [56]:
df = pd.DataFrame()

result = [parse_record_rdf(item) for item in records]
df_dnb_rdfxml = pd.DataFrame(result)
df_dnb_rdfxml_formatted = (df_dnb_rdfxml.style
       .format({'URN': make_clickable})
       .set_properties(**{'text-align': 'left'})
       .set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]) )
display(df_dnb_rdfxml_formatted)

with open('dnb_rdf_data.csv', 'w', encoding='utf-8') as f:
    df_dnb_rdfxml.to_csv(f, sep=';', index=False)

,IDN,CREATOR,TITLE,DATE,PUBLISHER,URN,ISBN
0,(DE-101)1039499449,N/A,Adath Jeshurun of West Bronx,1941,N/A,N/A,N/A
1,(DE-101)1039336116,N/A,Aus den Gemeinden,1950,N/A,N/A,N/A
2,(DE-101)1039504418,N/A,Aus den Gemeinden,1941,N/A,N/A,N/A
3,(DE-101)103908205X,N/A,Aus den Gemeinden,1946,N/A,N/A,N/A
4,(DE-101)1039273734,N/A,Aus den Gemeinden,1949,N/A,N/A,N/A
5,(DE-101)1039438938,N/A,Aus den Gemeinden,1940,N/A,N/A,N/A
6,(DE-101)1039320953,N/A,Aus den Gemeinden,1949,N/A,N/A,N/A
7,(DE-101)1039385184,N/A,B'nai Jeshurun beschliesst ihr Jubiläum,1950,N/A,N/A,N/A
8,(DE-101)1039376231,N/A,Congregation B'nai Jeshurun - 125 Jahre,1950,N/A,N/A,N/A
9,(DE-101)103949823X,N/A,K'Hal Adath Jeshurun,1941,N/A,N/A,N/A


# GND RDF-XML

In [93]:
query = ("per=Breuer, Isaak")
params = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query, 'recordSchema' : "RDFxml", 'maximumRecords': '100'}

request = requests.get("https://services.dnb.de/sru/authorities", params = params)
response = soup(request.content)
records = response.find_all('record')
print(records)


[<record><recordschema>RDFxml</recordschema><recordpacking>xml</recordpacking><recorddata><rdf:rdf xmlns:agrelon="https://d-nb.info/standards/elementset/agrelon#" xmlns:agrovoc="https://aims.fao.org/aos/agrovoc/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:bibo="http://purl.org/ontology/bibo/" xmlns:cidoc="http://www.cidoc-crm.org/cidoc-crm/" xmlns:dbp="http://dbpedia.org/property/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcmitype="http://purl.org/dc/dcmitype/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:dnb_intern="http://dnb.de/" xmlns:dnbt="https://d-nb.info/standards/elementset/dnb#" xmlns:ebu="http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#" xmlns:editeur="https://ns.editeur.org/thema/" xmlns:embne="https://datos.bne.es/resource/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:gbv="http://purl.org/ontology/gbv/" xmlns:geo="http://www.opengis.net/ont/geosparql#" xmlns:gndo="https://d-nb.info/standards/elementset/gnd#" xmlns:isbd="http://iflastandards.i

/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [94]:
result = [parse_record_gndrdf(item) for item in records]
df_gnd_rdfxml = pd.DataFrame(result)
df_gnd_rdfxml_formatted = (df_gnd_rdfxml.style
       .format({'LINK': make_clickable})
       .set_properties(**{'text-align': 'left'})
       .set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]) )
display(df_gnd_rdfxml_formatted)

with open('gnd_data.csv', 'w', encoding='utf-8') as f:
    df.to_csv(f, sep=';', index=False)

,IDN,NAME,TIME,LINK
0,1180782631,Messiasspuren,1918,https://d-nb.info/gnd/1180782631
1,118078393X,Jerusalem,1903,https://d-nb.info/gnd/118078393X
2,118852647,"Breuer, Isaak",1883-09-18-,https://d-nb.info/gnd/118852647
3,1223028585,Werke,N/A,https://d-nb.info/gnd/1223028585
